# Deploying and invoking Drift Detector Dataflow Flex template

In [3]:
import json
from datetime import datetime
import time
import googleapiclient.discovery


## Trigger a run of the template using REST API

In [8]:
template_path = 'gs://mlops-dev-workspace/dataflow-templates/log_analyzer.json'
location = 'us-central1'
project_id = 'mlops-dev-env'

In [9]:
service = googleapiclient.discovery.build('dataflow', 'v1b3')

parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_logs_tf',
    'model': 'covertype_tf',
    'version': 'v3',
    'start_time': '2020-06-03T17:00:00',
    'end_time': '2020-06-03T23:00:00',
    'output_path': 'gs://mlops-dev-workspace/drift-monitor/output/covertype-tf/test',
    'schema_file': 'gs://mlops-dev-workspace/drift-monitor/schema/schema.pbtxt',
    'baseline_stats_file': 'gs://mlops-dev-workspace/drift-monitor/baseline_stats/stats.pbtxt', 
    'time_window': '60m'
}
job_name = "log-analyzer-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

request = service.projects().locations().flexTemplates().launch(
    location=location,
    projectId=project_id,
    body=body)

response = request.execute()
response

{'job': {'id': '2020-06-04_15_07_41-10816826781620628839',
  'projectId': 'mlops-dev-env',
  'name': 'rr-log-analyzer-20200604-220740',
  'currentStateTime': '1970-01-01T00:00:00Z',
  'createTime': '2020-06-04T22:07:41.618675Z',
  'location': 'us-central1',
  'startTime': '2020-06-04T22:07:41.618675Z'}}

## Trigger a run of the template using REST API - Curl

In [ ]:
service_url = 'https://dataflow.googleapis.com/v1b3/projects/{}/locations/us-central1/flexTemplates:launch'.format(project_id)
headers_content = 'Content-Type: application/json'
access_token = !(gcloud auth print-access-token) 
headers_auth = 'Authorization: Bearer {}'.format(access_token[0])
parameters = {
    'request_response_log_table': 'mlops-dev-env.data_validation.covertype_logs_tf',
    'model': 'covertype_tf',
    'version': 'v3',
    'start_time': '2020-06-03T16:15:00',
    'end_time': '2020-06-03T22:51:00',
    'output_path': 'gs://mlops-dev-workspace/drift_monitor/output/tf/test2',
    'schema_file': 'gs://mlops-dev-workspace/drift_monitor/schema/schema.pbtxt',
    'time_window': '60m'

}
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
body = {
    'launch_parameter': 
         {
             'jobName': job_name,
             'parameters' : parameters,
             'containerSpecGcsPath': template_path
         }}

json_body = json.dumps(body)

In [ ]:
!curl -X POST \
  "{service_url}" \
  -H "{headers_content}" \
  -H "{headers_auth}" \
  -d '{json_body}'

## Trigger a run of the template using `gcloud`

In [ ]:
job_name = "data-drift-{}".format(time.strftime("%Y%m%d-%H%M%S"))
g_parameters = ','.join(['{}={}'.format(key,value) for key, value in parameters.items()])

!gcloud beta dataflow flex-template run {job_name} \
--template-file-gcs-location {template_path} \
--parameters {g_parameters}